# Umbrella sampling

In this exercise, we will introduce umbrella sampling. We will study a 1D potential energy surface described in [1] and used in [2]. The unperturbed free energy surface is described by,
$U_o(z) = (5z^3 - 10z + 3)z$.

[1] G. Hummer, in Free Energy Calculations, edited by C. Chipot and A.
Pohorille (Springer, Berlin, 2007), Vol. 86.

[2] D. Minh and A. Adib, Optimized Free Energies from Bidirectional Single-Molecule Force Spectroscopy, [Physical Review Letters 100(18): 180602 (2008)](https://link.aps.org/doi/10.1103/PhysRevLett.100.180602).

# Part 0 - Setting up the required software

The following cell will install pymbar, which is useful for the analysis of umbrella sampling.

In [ ]:
!pip install pymbar

In [ ]:
import numpy as np
import scipy
import scipy.integrate

import pymbar

%matplotlib inline
import matplotlib.pyplot as plt

# Part 1 - Monte Carlo Simulation

## Unperturbed surface

Let's first show the unperturbed surface and the expected probability of z based on the Boltzmann distribution.

In [ ]:
# The unperturbed surface
U_o = lambda z: (5*z*z*z - 10*z + 3)*z
z_min = -1.75
z_max = 1.5

# The partition function of the original surface
Q_o = scipy.integrate.quad(lambda z: np.exp(-U_o(z)), z_min, z_max)[0]
# The probability density
ρ_o = lambda z: np.exp(-U_o(z))/Q_o

In [ ]:
# Visualize the surface
z = np.linspace(z_min,z_max,51)

plt.subplot(1,2,1)
plt.tight_layout(w_pad=4)
plt.plot(z, U_o(z))
plt.title('Original free energy surface')
plt.xlabel('Position');
plt.ylabel('Free energy ($k_B T$)');

plt.subplot(1,2,2)
plt.plot(z, ρ_o(z))
plt.title('Probability density function')
plt.xlabel('Position');
plt.ylabel('Probability density');

The free energy surface has two local minima. The right well is much less probable than the left well.

Here are some quantitative properties of the surface:

In [ ]:
z_max_ρ_l = scipy.optimize.minimize(U_o, z_min)['x'][0]
U_z_max_ρ_l = U_o(z_max_ρ_l)
z_max_ρ_r = scipy.optimize.minimize(U_o, z_max)['x'][0]
U_z_max_ρ_r = U_o(z_max_ρ_r)

print(f'There is a minimum at {z_max_ρ_l:.4f} with energy {U_z_max_ρ_l:.4f}')
print(f'There is a minimum at {z_max_ρ_r:.4f} with energy {U_z_max_ρ_r:.4f}')
print(f'The free energy difference between the left and right well is {U_z_max_ρ_r - U_z_max_ρ_l:.4f}\n')

z_bar = scipy.integrate.quad(lambda z: z*ρ_o(z), z_min, z_max)[0]
std_z_bar = np.sqrt(scipy.integrate.quad(lambda z: z*z*ρ_o(z), z_min, z_max)[0] - z_bar*z_bar)
print(f"z has a mean of {z_bar:.4f} and standard deviation of {std_z_bar:.4f}")

## Sampling

Now we will use acceptance-rejection to generate independent configurations from the Boltzmann distribution.

In [ ]:
def acceptance_rejection(U, ρ_max, N, z_min, z_max, blocksize=5000):
  """
  Generates random samples from a 1D energy surface using
  the acceptance-rejection technique

  Parameters
  ----------
  U : function
    The potential energy
  ρ_max : float
    The maximum unnormalized probability
  N : int
    the number of samples to generate
  z_min : float
    the lower bound of the domain
  z_max : float
    the upper bound of the domain
  blocksize : int
    The number of uniform random variates to generate in an iteration.
  """
  z = []
  while len(z)<N:
    z_trial = np.random.uniform(z_min, z_max, blocksize)
    acc_trial = np.exp(-U(z_trial))/ρ_max
    z += list(z_trial[np.random.uniform(size=blocksize)<acc_trial])
  return np.array(z[:N])

Acceptance-rejection (in 1D) is like generating random points in 2D and keeping points that are "under" the probability density.

In [ ]:
# Illustration for acceptance-rejection
z_max_ρ = scipy.optimize.minimize(U_o, -1.1)['x'][0]
ρ_max = np.exp(-U_o(z_max_ρ))

plt.subplot(1,2,2)
plt.plot(z, np.exp(-U_o(z)))
plt.plot([z_min, z_min, z_max, z_max, z_min],[0, ρ_max, ρ_max, 0, 0])
plt.title('Unnormalized probability density function')
plt.xlabel('Position');
plt.ylabel('Probability density');

In [ ]:
N_samples = 1000

# Generate random samples from the unperturbed distribution
z_max_ρ = scipy.optimize.minimize(U_o, -1.1)['x'][0]
ρ_max = np.exp(-U_o(z_max_ρ))
z_random = acceptance_rejection(U_o, ρ_max, N_samples, z_min, z_max)

## Analysis

Next we will visualize this histogram of samples and the free energy surface.

In [ ]:
# Plot histogram of random samples and reconstructed free energy
(counts, edges) = np.histogram(z_random, bins=50, range=(z_min, z_max))
U_hat = -np.log(counts)

plt.subplot(1,2,1)
plt.tight_layout(w_pad=4)
plt.plot(z[:-1] + (z[1]-z[0])/2, counts)
plt.title('Histogram')
plt.xlabel('Position');
plt.ylabel('Counts');

plt.subplot(1,2,2)
plt.plot(z[:-1] + (z[1]-z[0])/2, U_hat)
plt.title('Reconstructed free energy surface')
plt.xlabel('Position');
plt.ylabel('Free energy ($k_B T$)');

# Estimate expectation values based on random samples
z_hat = np.mean(z_random)
std_z_hat = np.std(z_random)

ind_l = np.argmax(counts[z[:-1]<0])
ind_r = np.argmax(counts[z[:-1]>0]) + len(counts[z[:-1]<0])
print(f'There is a minimum at {z[ind_l]:.4f} with energy {U_hat[ind_l]:.4f}')
print(f'There is a minimum at {z[ind_r]:.4f} with energy {U_hat[ind_r]:.4f}')
print(f'The free energy difference between the left and right well is {U_hat[ind_r] - U_hat[ind_l]:.4f}\n')

print(f"z has an estimated mean of {z_hat:.4f} and estimated standard deviation of {std_z_hat:.4f}")

## Questions

--> If you use 1000 samples, do you observe many samples in both energy wells?

--> How do the following estimated quantities compare to their true values?
* free energy energy difference between the left and right wells
* standard deviation of z

--> Try estimating these properties based on 10000, 100000, and 1000000 samples. Describe how your estimates change as you increase the number of samples.

# Part 2 - Umbrella sampling

## Biased surface

Now let's look at what the energy surface and probability density function looks like with a harmonic biasing potential.

In [ ]:
N_windows = 20 # This is the number of thermodynamic states

# Potential energy of the system with a harmonic bias on z
def U_bias(z, k_s, z_o):
  return k_s*np.square(z-z_o)/2

def U(z, k_s, z_o):
  return U_o(z) + U_bias(z, k_s, z_o)

# Loop over multiple spring centers
plt.subplot(1,2,1)
plt.tight_layout(w_pad=4)
plt.title('Perturbed free energy surface')
plt.xlabel('Position');
plt.ylabel('Free energy ($k_B T$)');
for z_c in np.linspace(-1.5, 1.5, N_windows):
  plt.plot(z, U(z, 15, z_c))
plt.ylim(plt.ylim()[0], 40)

plt.subplot(1,2,2)
plt.title('Probability density function')
plt.xlabel('Position');
plt.ylabel('Free energy ($k_B T$)');
for z_c in np.linspace(-1.5, 1.5, N_windows):
  # The partition function of the perturbed surface
  Q_c = scipy.integrate.quad(lambda z: np.exp(-U(z, 15, z_c)), z_min, z_max)[0]
  # The probability density
  ρ_c = lambda z: np.exp(-U(z, 15, z_c))/Q_c
  plt.plot(z, ρ_c(z))

## Sampling

Now we will use the acceptance-rejection to generate independent configurations from the Boltzmann distribution with a harmonic bias and different spring constants.

In [ ]:
N_samples = 10000 # This is the total number of samples
z_random = []

for z_c in np.linspace(-1.5, 1.5, N_windows):
  # Generate random samples from the biased distribution
  z_max_ρ = scipy.optimize.minimize(lambda z: U(z, 15, z_c), z_c)['x'][0]
  ρ_max = np.exp(-U(z_max_ρ, 15, z_c))
  z_random_c = acceptance_rejection(lambda z: U(z, 15, z_c), ρ_max, \
    int(N_samples/N_windows), z_min, z_max)
  z_random.append(z_random_c)

## Analysis

We will visualize histogram of samples from *all* windows and free energy surfaces. Biased surfaces are obtained by the Boltzmann inverse and unbiased surfaces by subtracting the bias.

In [ ]:
figs = []
axs = []
for (ind, title, ylabel) in [(0,'Histogram','Counts'), \
  (1,'Reconstructed biased free energy surfaces','Free energy ($k_B T$)'), \
  (2,'Reconstructed free energy surfaces','Free energy ($k_B T$)')]:
  figs.append(plt.figure(ind))
  axs.append(figs[ind].add_axes((0,0,1,1)))
  plt.title(title)
  plt.xlabel('Position')
  plt.ylabel(ylabel)

for (window_index, z_c) in enumerate(np.linspace(-1.5, 1.5, N_windows)):
  # Plot histogram of random samples and reconstructed biased free energy
  (counts, edges) = np.histogram(z_random[window_index], bins=50, range=(z_min, z_max))
  U_hat = -np.log(counts)

  bin_centers = z[:-1] + (z[1]-z[0])/2
  axs[0].plot(bin_centers, counts)
  axs[1].plot(bin_centers, U_hat)
  axs[2].plot(bin_centers, U_hat - U_bias(bin_centers, 15, z_c))

After removing the effects of bias, free energy surfaces from each window contain only *part* of the total free energy surface. To obtain a complete free energy surface, we need to "piece together" information from each window. This entails shifting up and down segments of the surface based on their relative free energies. The traditional way to perform this shifting and calculate relative free energies is called the weighted histogram analysis method (WHAM), which makes use of histograms like we have visualized [1]. The multi-state Bennett Acceptance Ratio is a statistical optimal estimator that can be thought of as a binless version of WHAM [2].

In [ ]:
# Set up variables to use MBAR to estimate free energies
z_random_flat = np.array(z_random).flatten()

# Calculate the biasing energy for every sample in every state
u_kn = []
N_k = []
for z_c in np.linspace(-1.5, 1.5, N_windows):
  u_kn.append(U_bias(z_random_flat, 15, z_c))
  N_k.append(int(N_samples/N_windows))
u_kn = np.array(u_kn)
N_k = np.array(N_k)

# The reduced potential of the unperturbed state
u_n = np.zeros(z_random_flat.shape)
# Compute free energy surface based on Guassian kernel density estimate
fes = pymbar.FES(u_kn, N_k)
fes.generate_fes(u_n, z_random_flat, fes_type='kde', kde_parameters={'bandwidth':0.05})

f_i = fes.get_fes(z)['f_i']
mbar = fes.get_mbar()

# Plot the free energy
# of the perturbed states
plt.figure()
plt.plot(np.linspace(-1.5, 1.5, N_windows), mbar.f_k, '.-')
plt.title('Free energy of perturbed system')
plt.xlabel('Bias center, $z_o$')
plt.ylabel('Free energy ($k_BT$)')

# of the unperturbed system
plt.figure()
plt.plot(z, f_i - np.min(f_i))
plt.plot(z, U_o(z) - np.min(U_o(z)), 'k')
plt.title('Free energy of unperturbed system')
plt.xlabel('Position, $z$')
plt.ylabel('Free energy ($k_BT$)')
plt.legend(['Reconstructed','Original'])

ind_l = np.argmin(f_i[z<0])
ind_r = np.argmin(f_i[z>0]) + len(f_i[z<0])
print(f'There is a minimum at {z[ind_l]:.4f} with energy {f_i[ind_l]:.4f}')
print(f'There is a minimum at {z[ind_r]:.4f} with energy {f_i[ind_r]:.4f}')
print(f'The free energy difference between the left and right well is {f_i[ind_r] - f_i[ind_l]:.4f}\n')

# Compute mean annd expectation value
A_in = np.array([z_random_flat,z_random_flat**2])
u_n = np.zeros(z_random_flat.shape) # The reduced potential of the state of interest is unperturbed
results = mbar.compute_multiple_expectations(A_in, u_n)

z_hat = results['mu'][0]
std_z_hat = np.sqrt(results['mu'][1] - results['mu'][0]**2)

print(f"z has an estimated mean of {z_hat:.4f} and estimated standard deviation of {std_z_hat:.4f}")

## Questions

--> If you use 1000 samples, do you observe many samples in both energy wells?

--> How do the following estimated quantities compare to their true values?
* free energy energy difference between the left and right wells
* standard deviation of z

--> Try estimating these properties based on 10000, 100000, and 1000000 samples. Describe how your estimates change as you increase the number of samples.

# References

[1] Kumar, S.; Bouzida, D.; Swendsen, R. H.; Kollman, P. A.; Rosenberg, J. M. J. Comput. Chem. 1992, 13, 1011-1021. \\
[2] Shirts, M. R.; Chodera, J. D. Statistically Optimal Analysis of Samples from Multiple Equilibrium States. The Journal of Chemical Physics 2008, 129 (12), 124105. https://doi.org/10.1063/1.2978177.
